Esse notebook foi feito para gerar mel-espectrogramas usando a biblioteca librosa.

In [ ]:
#Importando biblioteca utilizadas.

import os
import librosa as librosa
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display
import warnings
import cv2
import timeit
import pandas as pd

warnings.filterwarnings('ignore')

A função abaixo recebe um mel-espectrograma gerado pelo librosa e retorna uma array que representa uma imagem. Será usado mais abaixo.

In [ ]:
def mono_to_color(X: np.ndarray,
                  mean=None,
                  std=None,
                  norm_max=None,
                  norm_min=None,
                  eps=1e-6):
    """
    Code from https://www.kaggle.com/daisukelab/creating-fat2019-preprocessed-data
    """
    # Stack X as [X,X,X]
    #X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

O código em baixo é uma maneira mais rápida de carregar os audios com o librosa

In [ ]:
def get_clip_sr(path,offset=0,duration=None):
    clip, sr_native = librosa.core.audio.__audioread_load(path, offset=offset, duration=duration, dtype=np.float32)
    clip = librosa.to_mono(clip)
    sr = 22050
    if sr_native > 0:
        clip = librosa.resample(clip, sr_native, sr, res_type='kaiser_fast')
    return clip, sr

Abaixo estou iterando por todos as linhas presentes no arquivo train.csv e extraindo os mel-espectrogramas e gerando imagens para os 5s iniciais de cada audios pertencente a primeira espécie.

In [ ]:
#Definindo os caminhos dos audios e abrindo o arquivo csv
dir = "../input/segmentation-by-amplitude-signal-on-subsample/"
sets = ['val','test','train']

t=0
v=0
e=0
cnt = 0 

for _set in sets:
    
    path_set = os.path.join(dir,_set) 
    species = os.listdir(path_set)
    print(len(species))
    
    os.mkdir(_set);
    
    species.sort()
    
    for sp in species:
               
        os.mkdir(os.path.join(_set,sp));        
                
        path_species = os.path.join(path_set,sp)
        
        if(not(os.path.isdir(path_species))):
            continue
        
        audios = os.listdir(path_species)
        
        for audio in audios:
            if(_set == 'train'):
                t+=1;
                
            if(_set == 'val'):
                v+=1;
            
            if(_set == 'test'):
                e+=1;

            full_path = os.path.join(path_species,audio)
            
            #Carregando os 5s iniciais do aúdio
            y,sr =  get_clip_sr(full_path);

            #Transformando em mel-spectrograma
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=310,fmin=160,fmax=10300)
            S_dB = librosa.power_to_db(S, ref=np.max)

            #Ajustando imagem
            im = mono_to_color(S_dB);
            im = cv2.resize(im, (224,224))
            im = cv2.flip(im, 0)

            #Escrevendo imagem
            cv2.imwrite('{}/{}/{}.png'.format(_set,sp,audio[:-4]),im)
            cnt += 1
            
            
print('{} samples for train'.format(t))
print('{} samples for validation'.format(v))
print('{} samples for test'.format(e))

Exemplo de visualização das imagens geradas.

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('./test/aldfly/6.png')
imgplot = plt.imshow(img)